In [4]:
import pandas as pd
import os, datetime, re, csv

## Test_

In [ ]:
# root_dir = "C:/Users/LGCARE/Desktop/zip파일"

# try:
#     for item in os.listdir(root_dir):
#         print(item)
# except PermissionError:
#     print("폴더 접근 권한이 없습니다. 관리자 권한으로 실행해 주세요.")

In [ ]:
# root_dir = "C:/Users/LGCARE/Desktop/zip파일/누끼컷 모음"

# try:
#     for item in os.listdir(root_dir):
#         print(item)
# except PermissionError:
#     print("폴더 접근 권한이 없습니다. 관리자 권한으로 실행해 주세요.")

In [ ]:
# df = pd.read_csv('C:/Users/LGCARE/Desktop/zip파일/naver_drive.csv', encoding= 'cp949')
# df.head()

In [ ]:
import os, re, datetime
import pandas as pd

In [ ]:
# 1. 네이버 드라이브 내 폴더 문서 번호 부여

def generate_doc_dataframe(target_directory, start_doc_num=1):
    """
    지정된 디렉토리 내 모든 파일에 대해 문서 번호를 부여하고
    파일 정보를 포함한 데이터프레임을 반환

    Parameters:
    - target_directory (str): 검색할 폴더 경로
    - start_doc_num (int): 문서 번호 시작 값 (예: 1이면 DOC00001부터 시작)

    Returns:
    - pandas.DataFrame: 열 - 문서 번호, (구)파일명, 확장자, 최종 업로드일, 용량, 폴더 경로
    """
    data = []
    doc_counter = start_doc_num

    for root, _, files in os.walk(target_directory):
        for file in sorted(files):
            file_path = os.path.join(root, file)
            file_name, file_ext = os.path.splitext(file)
            mod_time = datetime.datetime.fromtimestamp(os.path.getmtime(file_path)).strftime('%Y-%m-%d')

            # 파일 크기 (바이트 기준 → KB로 변환, 소수점 둘째 자리까지)
            file_size_kb = os.path.getsize(file_path) / (1024 ** 2)
            file_size_str = f"{file_size_kb:.2f} MB"

            doc_num = f'DOC{doc_counter:05}'  # 다섯 자리 문서번호
            doc_counter += 1

            data.append({
                '문서 번호': doc_num,
                '(구)파일명': file,
                '확장자': file_ext,
                '최종 업로드일': mod_time,
                '용량': file_size_str,
                '폴더 경로': root
            })

    df = pd.DataFrame(data)
    print(f"✅ 총 {len(df)}개의 파일이 문서 번호와 함께 데이터프레임으로 저장되었습니다.")
    return df

In [20]:
df = generate_doc_dataframe('C:/Users/LGCARE/Desktop/zip파일/누끼컷 모음', start_doc_num= 1)
df.tail()

✅ 총 1030개의 파일이 문서 번호와 함께 데이터프레임으로 저장되었습니다.


,문서 번호,(구)파일명,확장자,최종 업로드일,용량,폴더 경로
1025,DOC01026,DOC00990_0320__[빌리프] 프솔 비건 스팟패치 누끼컷_세트_Q 출고데이터...,.psd,2024-03-20,168.14 MB,C:/Users/LGCARE/Desktop/zip파일/누끼컷 모음\프라블럼솔루션 R...
1026,DOC01027,DOC00991_0320__[빌리프] 프솔 비건 스팟패치 누끼컷_파우치_Q 출고데이...,.jpg,2024-03-19,0.88 MB,C:/Users/LGCARE/Desktop/zip파일/누끼컷 모음\프라블럼솔루션 R...
1027,DOC01028,DOC00992_0320__[빌리프] 프솔 비건 스팟패치 누끼컷_파우치_Q 출고데이...,.psd,2024-03-20,54.52 MB,C:/Users/LGCARE/Desktop/zip파일/누끼컷 모음\프라블럼솔루션 R...
1028,DOC01029,DOC00993_0320__[빌리프] 프솔 비건 스팟패치 누끼컷_필름_Q 출고데이터...,.jpg,2024-03-12,0.47 MB,C:/Users/LGCARE/Desktop/zip파일/누끼컷 모음\프라블럼솔루션 R...
1029,DOC01030,DOC00994_0320__[빌리프] 프솔 비건 스팟패치 누끼컷_필름_Q 출고데이터...,.psd,2024-03-20,26.07 MB,C:/Users/LGCARE/Desktop/zip파일/누끼컷 모음\프라블럼솔루션 R...


In [ ]:
# 2. 네이버 드라이브 폴더 내 파일명 변환

# 문서 번호_파일명

def rename_files_with_DocNum(self, df, naver_drive_directory):
    """
    주어진 데이터프레임을 기준으로 파일명을 '문서번호_파일명' 형식으로 변경

    Parameters:
    - df (pd.DataFrame): generate_doc_dataframe 함수에서 생성된 데이터프레임
    - base_directory (str): 변경할 파일이 존재하는 폴더 경로
    """
    for idx, row in df.iterrows():
        doc_num = str(row['문서 번호']).strip() if pd.notnull(row['문서 번호']) else ''
        file_name = str(row['(구)파일명']).strip() if pd.notnull(row['(구)파일명']) else ''
        expected_filename = file_name

        for root, dirs, files in os.walk(naver_drive_directory):
            for f in files:
                if f.strip() == expected_filename:
                    original_path = os.path.join(root, f)
                    new_filename = f"{doc_num}_{f}"
                    new_path = os.path.join(root, new_filename)

                    if not os.path.exists(new_path):
                        os.rename(original_path, new_path)
                        print(f"✅ Renamed: {original_path} -> {new_path}")
                    else:
                        print(f"⚠️ Skipped (already exists): {new_path}") # 이름이 동일한 경우

In [ ]:
rename_files_with_DocNum("C:/Users/LGCARE/Desktop/zip파일/naver_drive.csv", 
                               "C:/Users/LGCARE/Desktop/zip파일/누끼컷 모음")


In [ ]:
# 3. 로컬 이사 완료 후 변수명 변경

# 문서 번호_제품명_용량_상위 폴더명_최종 수정일

class Rename() :
  def __init__(self, base_directory_path):
    self.base_path = base_directory_path

  def __rename_files_Help(self, product_name_folder_path, product_name):
    """ 문서번호_제품명_용량_상위폴더명_최종수정일 형식으로 변경 """

    def extract_volume_info(file_name):
        """ 파일명에서 '100ml', '200g' 같은 정보를 추출 """
        match = re.search(r'(\d+(?:ml|g))', file_name)
        return match.group(1) if match else ""

    def extract_country_keyword(file_name):
            """ 파일명에서 사용 국가 키워드 추출 """
            keywords = ['중국용', '국내중국겸용', '국내용', '중국']  # 우선순위 높은 순서로 정렬
            for keyword in keywords:
                if keyword in file_name:
                    return f"_{keyword}"
            return ""
    
    file_dates = {}  # 중복 방지용 딕셔너리

    for root, _, files in os.walk(product_name_folder_path):
        folder_name = os.path.basename(root)

        for file in sorted(files):
            src_path = os.path.join(root, file)
            mod_time = datetime.datetime.fromtimestamp(os.path.getmtime(src_path)).strftime('%Y%m%d')

            # 기존 파일명에서 DOCx_ 문서번호 추출
            doc_match = re.match(r'(DOC\d+)_', file)
            doc_number = doc_match.group(1) if doc_match else ""

            # 파일명에서 용량 정보 추출
            volume_info = extract_volume_info(file)
            volume_suffix = f"_{volume_info}" if volume_info else ""

            # 사용 국가 키워드 추출
            country_suffix = extract_country_keyword(file)

            # 새로운 파일명 생성
            new_name = f"{doc_number}_{product_name}{volume_suffix}_{folder_name}_{mod_time}{country_suffix}"

            ext = os.path.splitext(file)[1]
            count = file_dates.get(new_name, 0) + 1
            file_dates[new_name] = count
            new_file_name = f"{new_name}_{count}{ext}" if count > 1 else f"{new_name}{ext}"

            new_path = os.path.join(root, new_file_name)
            os.rename(src_path, new_path)
  
  def rename_all_product_folders(self):
    for product_line in os.listdir(self.base_path):
        product_line_path = os.path.join(self.base_path, product_line)

        # "0_BrandAsset" 폴더는 건너뛰기
        if product_line == "0_BrandAsset":
            print(f"Skipping brand asset folder: {product_line}")
            continue

        # ProductLine 폴더 내 ProductName 폴더 찾기
        if os.path.isdir(product_line_path):
            for product_name in os.listdir(product_line_path):
                product_name_path = os.path.join(product_line_path, product_name)
                print(f"Renaming: {product_name}")
                self.__rename_files_Help(product_name_path, product_name)
  
  def __remove_docnumber_from_filenames(self, root_dir):
    """
    파일명 앞의 문서번호 (예: DOC001_) 를 제거하고 이름 변경
    """
    renamed_count = 0

    for folder_path, _, files in os.walk(root_dir):
        for file in files:
            # 정규표현식으로 DOCxxx_ 패턴 추출
            match = re.match(r'^(DOC\d+_)(.+)', file)
            if match:
                doc_prefix, rest_of_name = match.groups()
                old_path = os.path.join(folder_path, file)
                new_path = os.path.join(folder_path, rest_of_name)

                # 같은 이름 파일 존재 시 덮어쓰기 방지
                if not os.path.exists(new_path):
                    os.rename(old_path, new_path)
                    renamed_count += 1
                    print(f"🔁 Renamed: {file} → {rest_of_name}")
                else:
                    print(f"⚠️ 이미 존재해서 건너뜀: {rest_of_name}")
    
    print(f"✅ 문서번호 제거 완료: 총 {renamed_count}개 파일 이름 변경됨")

  def remove_docnum_all_product_folders(self):
      for product_line in os.listdir(self.base_path):
        product_line_path = os.path.join(self.base_path, product_line)
        
        # "0_BrandAsset" 폴더는 건너뛰기
        if product_line == "0_BrandAsset":
            print(f"Skipping brand asset folder: {product_line}")
            continue
        
        # ProductLine 폴더 내 ProductName 폴더 찾기
        if os.path.isdir(product_line_path):
            for product_name in os.listdir(product_line_path):
                product_name_path = os.path.join(product_line_path, product_name)
                print(f"Removing_DocNum: {product_name}")
                self.__remove_docnumber_from_filenames(product_name_path)

In [11]:
rename = Rename(base_directory_path= 'D:/01. 빌리프_컨텐츠아카이빙/Belif_작업중_아밤테스트')
rename.rename_all_product_folders()

Renaming: 2D
Renaming: 3D
Renaming: Cutout
Renaming: Logo
Renaming: BarrierEmulsion
Renaming: BarrierToner
Renaming: Bomb
Renaming: EyeSerum
Renaming: VCream
Renaming: VCreamAD
Renaming: VEyeSerum
Renaming: WaterEssence
Renaming: AquaBomb
Renaming: AquaBomb_원본
Renaming: BrighteningAmpouleSerum
Renaming: CoolingSunstick
Renaming: EyeGel
Renaming: JellyCleanser
Renaming: Mist
Renaming: RefiningAmpouleSerum
Renaming: SleepingMask
Renaming: SmartCleansingOilBalm
Renaming: SunSerum
Renaming: SunSerumAD
Renaming: VitaWaterCream
Renaming: BergamotHerbalExtractToner
Renaming: ClassicCreamUltimate
Renaming: ClassicEssenceIncrement
Renaming: CreamyMoisturizerDeepMoist
Renaming: EucalyptusHerbalExtractToner
Renaming: HungarianWaterEssence
Renaming: HungarianWaterEssenceMoistureBinder
Renaming: MilkyMoisturizerHydraBalancing
Renaming: Numero10Essence
Renaming: Numero10Mist
Renaming: OilControlMoisturizerFresh
Renaming: WitchHazelHerbalExtractToner
Renaming: BodyBomba
Renaming: InTheRoseGardenHandC

In [15]:
rename = Rename(base_directory_path= 'C:/Users/LGCARE/Desktop/zip파일/Belif_작업중')
rename.rename_all_product_folders()

Renaming: 2D
Renaming: 3D
Renaming: Cutout
Renaming: Logo
Renaming: BarrierEmulsion
Renaming: BarrierToner
Renaming: Bomb
Renaming: EyeSerum
Renaming: VCream
Renaming: VCreamAD
Renaming: VEyeSerum
Renaming: WaterEssence
Renaming: AquaBomb
Renaming: BrighteningAmpouleSerum
Renaming: CoolingSunstick
Renaming: EyeGel
Renaming: JellyCleanser
Renaming: Mist
Renaming: RefiningAmpouleSerum
Renaming: SleepingMask
Renaming: SmartCleansingOilBalm
Renaming: SunSerum
Renaming: SunSerumAD
Renaming: VitaWaterCream
Renaming: BergamotHerbalExtractToner
Renaming: ClassicCreamUltimate
Renaming: ClassicEssenceIncrement
Renaming: CreamyMoisturizerDeepMoist
Renaming: EucalyptusHerbalExtractToner
Renaming: HungarianWaterEssence
Renaming: HungarianWaterEssenceMoistureBinder
Renaming: MilkyMoisturizerHydraBalancing
Renaming: Numero10Essence
Renaming: Numero10Mist
Renaming: OilControlMoisturizerFresh
Renaming: WitchHazelHerbalExtractToner
Renaming: BodyBomba
Renaming: InTheRoseGardenHandCream
Renaming: Lavende

In [ ]:
# 4. 기존 csv파일과 문서 번호 매치한 병합 csv파일 생성
# - 이사한 데이터 문서 번호 / 파일명 / 폴더 경로 로 csv 파일 생성

# ** 매치되지 않은 파일은 그대로 > 이사 제외 사유 기입

def foldertree_to_csv_merge_by_doc(existing_csv_path, root_dir, output_csv_path=None):
    """
    기존 CSV 파일과 폴더 정보(문서번호/파일명/폴더경로)를 '문서번호' 기준으로 병합하여 저장
    """
    # 기존 CSV 불러오기
    df_existing = pd.read_csv(existing_csv_path, encoding="cp949")
    
    file_data = []
    for product_line in os.listdir(root_dir):
        product_line_path = os.path.join(root_dir, product_line)
        if os.path.isdir(product_line_path):
            for product_name in os.listdir(product_line_path):
                product_name_path = os.path.join(product_line_path, product_name)
                for folder_path, _, files in os.walk(product_name_path):
                    for file in files:
                        file_name, ext = os.path.splitext(file)
                        parts = file_name.split("_", 1)
                        if len(parts) >= 2:
                            doc_number = parts[0]
                            real_name = "_".join(parts[1:]) + ext
                        else:
                            doc_number = ""
                            real_name = file
                        file_data.append([doc_number, real_name, folder_path])
    
    # 새로운 폴더 기반 데이터프레임 생성
    df_folder = pd.DataFrame(file_data, columns=["문서 번호_구글", "(신)파일명", "폴더 경로"])

    # 병합: 문서번호 기준으로 left join
    df_merged = pd.merge(df_existing, df_folder, left_on="문서 번호", right_on= '문서 번호_구글', how="left")
    
    # 저장 경로 지정
    if not output_csv_path:
        output_csv_path = os.path.join(root_dir, "merged_naver_google.csv")

    df_merged.to_csv(output_csv_path, index=False, encoding="cp949")
    print(f"✅ 병합된 CSV 파일 생성 완료: {output_csv_path}")

In [74]:
# 검색할 디렉터리 설정
fordertree_path = "C:/Users/LGCARE/Desktop/zip파일/naver_drive.csv"  # 원하는 폴더 경로로 변경하세요
root_dir = "C:/Users/LGCARE/Desktop/zip파일/Belif_작업중"
output_csv_path = "C:/Users/LGCARE/Desktop/zip파일/merged_naver_google.csv"
foldertree_to_csv_merge_by_doc(fordertree_path, root_dir, output_csv_path)

✅ 병합된 CSV 파일 생성 완료: C:/Users/LGCARE/Desktop/zip파일/merged_naver_google.csv


In [75]:
# __________________실행 확인

In [20]:
rename = Rename(base_directory_path= 'C:/Users/LGCARE/Desktop/zip파일/Belif_작업중')
rename.remove_docnum_all_product_folders()

Removing_DocNum: 2D
✅ 문서번호 제거 완료: 총 0개 파일 이름 변경됨
Removing_DocNum: 3D
✅ 문서번호 제거 완료: 총 0개 파일 이름 변경됨
Removing_DocNum: Cutout
✅ 문서번호 제거 완료: 총 0개 파일 이름 변경됨
Removing_DocNum: Logo
✅ 문서번호 제거 완료: 총 0개 파일 이름 변경됨
Removing_DocNum: BarrierEmulsion
✅ 문서번호 제거 완료: 총 0개 파일 이름 변경됨
Removing_DocNum: BarrierToner
✅ 문서번호 제거 완료: 총 0개 파일 이름 변경됨
Removing_DocNum: Bomb
✅ 문서번호 제거 완료: 총 0개 파일 이름 변경됨
Removing_DocNum: EyeSerum
✅ 문서번호 제거 완료: 총 0개 파일 이름 변경됨
Removing_DocNum: VCream
✅ 문서번호 제거 완료: 총 0개 파일 이름 변경됨
Removing_DocNum: VCreamAD
✅ 문서번호 제거 완료: 총 0개 파일 이름 변경됨
Removing_DocNum: VEyeSerum
✅ 문서번호 제거 완료: 총 0개 파일 이름 변경됨
Removing_DocNum: WaterEssence
✅ 문서번호 제거 완료: 총 0개 파일 이름 변경됨
Removing_DocNum: AquaBomb
✅ 문서번호 제거 완료: 총 0개 파일 이름 변경됨
Removing_DocNum: BrighteningAmpouleSerum
✅ 문서번호 제거 완료: 총 0개 파일 이름 변경됨
Removing_DocNum: CoolingSunstick
✅ 문서번호 제거 완료: 총 0개 파일 이름 변경됨
Removing_DocNum: EyeGel
✅ 문서번호 제거 완료: 총 0개 파일 이름 변경됨
Removing_DocNum: JellyCleanser
✅ 문서번호 제거 완료: 총 0개 파일 이름 변경됨
Removing_DocNum: Mist
✅ 문서번호 제거 완료: 총 0개 파일 이름 변

## Test_250404

- assign_DocNum_and_DataFrame Test

In [1]:
from DigitalContentsArchiving import DigitalContentsArchiving

digitalcontentsarchiving = DigitalContentsArchiving('D:/00. Test/Belif')

In [ ]:
naver_drive_df = digitalcontentsarchiving.assign_DocNum_Help('D:/00. Test/ABO 제작 콘텐츠', start_doc_number= 1)

naver_drive_df.info()
# 두 번 하면 df가 생성이 안됨
# assign - df생성 함수 삭제

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame


In [7]:
naver_drive_df.head()

""
